In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Load the dataset
file_path = '/content/drive/MyDrive/Farm Project/Release/models/Feed Pattern /nutririon_data_updated.xlsx'
df = pd.read_excel(file_path)

# Display the first few rows
df.head()

,cattle_id,cattle_name,cattle_breed,farm,place_nu,health_status,status,feeding_pattern_id,feeding_time_morning,food_type_morning,...,food_type_evening,feeding_amount_KG_evening,score_evening,nutrition_info.2,feed_platform,feed_bunk,feeding_amount_KG||L,travel_distance_per_day_KM,farmers_id,farmer_name
0,1,A1,Jersey,F1,F1-L-1,Healthy,Lactating,FP_1,2024-09-24 08:00:00,Coconut Poonac,...,"Paddy Straw, Legumes",3.0,2.0,"High-fiber, low protein, low energy",cement,A1-bunk,3.666667,Napp,"FAR-118-20 , FAR-132-20, FAR-135-20","Gamage , Ratna, Keerthi"
1,2,A2,Jersey,F1,F1-L-2,Healthy,Lactating,FP_1,2024-09-24 08:00:00,Coconut Poonac,...,"Paddy Straw, Legumes",3.0,2.0,"High-fiber, low protein, low energy",cement,A-bunk,3.666667,Napp,"FAR-118-20 , FAR-132-20, FAR-135-20","Gamage , Ratna, Keerthi"
2,3,A3,Jersey,F1,F1-L-3,Healthy,Lactating,FP_1,2024-09-24 08:00:00,Coconut Poonac,...,"Paddy Straw, Legumes",3.0,2.0,"High-fiber, low protein, low energy",cement,A-bunk,3.666667,Napp,"FAR-118-20 , FAR-132-20, FAR-135-20","Gamage , Ratna, Keerthi"
3,4,A4,Jersey,F1,F1-L-4,Healthy,Lactating,FP_1,2024-09-24 08:00:00,Coconut Poonac,...,"Paddy Straw, Legumes",3.0,2.0,"High-fiber, low protein, low energy",cement,A-bunk,3.666667,Napp,"FAR-118-20 , FAR-132-20, FAR-135-20","Gamage , Ratna, Keerthi"
4,5,A5,Jersey,F1,F1-L-5,Healthy,Lactating,FP_1,2024-09-24 08:00:00,Coconut Poonac,...,"Paddy Straw, Legumes",3.0,2.0,"High-fiber, low protein, low energy",cement,A-bunk,3.666667,Napp,"FAR-118-20 , FAR-132-20, FAR-135-20","Gamage , Ratna, Keerthi"


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   cattle_id                   360 non-null    int64         
 1   cattle_name                 360 non-null    object        
 2   cattle_breed                360 non-null    object        
 3   farm                        360 non-null    object        
 4   place_nu                    360 non-null    object        
 5   health_status               360 non-null    object        
 6   status                      360 non-null    object        
 7   feeding_pattern_id          360 non-null    object        
 8   feeding_time_morning        360 non-null    datetime64[ns]
 9   food_type_morning           360 non-null    object        
 10  feeding_amount_KG_morning   300 non-null    float64       
 11  score_morning               300 non-null    float64       

In [3]:
# List the columns you want to remove
columns_to_remove = ['cattle_id', 'cattle_name', 'farm', 'place_nu', 'feeding_pattern_id', 'feeding_time_morning', 'nutrition_info', 'feeding_time_noon', 'nutrition_info.1', 'feeding_time_evening', 'nutrition_info.2', 'feed_bunk', 'feed_platform', 'feeding_amount_KG||L', 'farmers_id', 'farmer_name']

# Drop the columns
df = df.drop(columns=columns_to_remove)

# Display the updated DataFrame
df.head()


,cattle_breed,health_status,status,food_type_morning,feeding_amount_KG_morning,score_morning,food_type_noon,feeding_amount_KG_noon,score_noon,food_type_evening,feeding_amount_KG_evening,score_evening,travel_distance_per_day_KM
0,Jersey,Healthy,Lactating,Coconut Poonac,3.0,2.0,"Napier Grass, Guinea grass",5.0,2.0,"Paddy Straw, Legumes",3.0,2.0,Napp
1,Jersey,Healthy,Lactating,Coconut Poonac,3.0,2.0,"Napier Grass, Guinea grass",5.0,2.0,"Paddy Straw, Legumes",3.0,2.0,Napp
2,Jersey,Healthy,Lactating,Coconut Poonac,3.0,3.0,"Napier Grass, Guinea grass",5.0,2.0,"Paddy Straw, Legumes",3.0,2.0,Napp
3,Jersey,Healthy,Lactating,Coconut Poonac,3.0,2.0,"Napier Grass, Guinea grass",5.0,2.0,"Paddy Straw, Legumes",3.0,2.0,Napp
4,Jersey,Healthy,Lactating,Coconut Poonac,3.0,2.0,"Napier Grass, Guinea grass",5.0,2.0,"Paddy Straw, Legumes",3.0,2.0,Napp


In [4]:
from sklearn.preprocessing import LabelEncoder

# List of columns to label encode
columns_to_encode = ['cattle_breed', 'health_status', 'status', 'food_type_morning', 'food_type_noon', 'food_type_evening']

# Initialize LabelEncoder
encoder = LabelEncoder()

# Create a dictionary to store encoding values
encoding_dict = {}

# Apply label encoding to each column
for column in columns_to_encode:
    df[column] = encoder.fit_transform(df[column])
    encoding_dict[column] = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))

    # Print the indexes of encodings for the current column
    print(f"Encoded values for '{column}': {encoding_dict[column]}")

# Display the updated DataFrame
df.head()


Encoded values for 'cattle_breed': {'Australian Milking Zebu': 0, 'Ayrshire': 1, 'Friesian': 2, 'Jersey': 3, 'Lanka White': 4, 'Sahiwal': 5}
Encoded values for 'health_status': {'Healthy': 0, 'Sick': 1}
Encoded values for 'status': {'Breeding': 0, 'Bulls': 1, 'Calves ': 2, 'Heifers ': 3, 'Lactating': 4, 'Pregnant': 5}
Encoded values for 'food_type_morning': {'Coconut Poonac': 0, 'Coconut Poonac, Grass': 1, 'Milk': 2}
Encoded values for 'food_type_noon': {'Coconut Poonac, Grass': 0, 'Grass, Paddy Straw': 1, 'Napier Grass, Guinea grass': 2, 'Napier Grass, Guinea grass, Para grass': 3, 'Napier Grass, Guinea grass,Gliricidia': 4, 'Paddy Straw, Grass (Chopped)': 5, 'Para grass, Gliricidia': 6}
Encoded values for 'food_type_evening': {'Milk': 0, 'Paddy Straw': 1, 'Paddy Straw, Corn': 2, 'Paddy Straw, Grass': 3, 'Paddy Straw, Legumes': 4}


,cattle_breed,health_status,status,food_type_morning,feeding_amount_KG_morning,score_morning,food_type_noon,feeding_amount_KG_noon,score_noon,food_type_evening,feeding_amount_KG_evening,score_evening,travel_distance_per_day_KM
0,3,0,4,0,3.0,2.0,2,5.0,2.0,4,3.0,2.0,Napp
1,3,0,4,0,3.0,2.0,2,5.0,2.0,4,3.0,2.0,Napp
2,3,0,4,0,3.0,3.0,2,5.0,2.0,4,3.0,2.0,Napp
3,3,0,4,0,3.0,2.0,2,5.0,2.0,4,3.0,2.0,Napp
4,3,0,4,0,3.0,2.0,2,5.0,2.0,4,3.0,2.0,Napp


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   cattle_breed                360 non-null    int64  
 1   health_status               360 non-null    int64  
 2   status                      360 non-null    int64  
 3   feeding_amount_KG_morning   300 non-null    float64
 4   score_morning               300 non-null    float64
 5   feeding_amount_KG_noon      300 non-null    float64
 6   score_noon                  300 non-null    float64
 7   feeding_amount_KG_evening   300 non-null    float64
 8   score_evening               300 non-null    float64
 9   travel_distance_per_day_KM  360 non-null    float64
 10  food_type                   360 non-null    object 
dtypes: float64(7), int64(3), object(1)
memory usage: 31.1+ KB


In [7]:
df['travel_distance_per_day_KM'] = pd.to_numeric(df['travel_distance_per_day_KM'], errors='coerce')

# Replace with 0
df['travel_distance_per_day_KM'].fillna(0, inplace=True)

# Display the updated DataFrame
df.head()

<ipython-input-7-25a51353d7c9>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['travel_distance_per_day_KM'].fillna(0, inplace=True)


,cattle_breed,health_status,status,food_type_morning,feeding_amount_KG_morning,score_morning,food_type_noon,feeding_amount_KG_noon,score_noon,food_type_evening,feeding_amount_KG_evening,score_evening,travel_distance_per_day_KM
0,3,0,4,0,3.0,2.0,2,5.0,2.0,4,3.0,2.0,0.0
1,3,0,4,0,3.0,2.0,2,5.0,2.0,4,3.0,2.0,0.0
2,3,0,4,0,3.0,3.0,2,5.0,2.0,4,3.0,2.0,0.0
3,3,0,4,0,3.0,2.0,2,5.0,2.0,4,3.0,2.0,0.0
4,3,0,4,0,3.0,2.0,2,5.0,2.0,4,3.0,2.0,0.0


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Step 1: Combine the food type columns into one
df['food_type'] = df['food_type_morning'].astype(str) + '-' + df['food_type_noon'].astype(str) + '-' + df['food_type_evening'].astype(str)

# Step 2: Drop the original food type columns
df = df.drop(columns=['food_type_morning', 'food_type_noon', 'food_type_evening'])

In [10]:
X = df.drop(columns=['food_type'])
y = df['food_type']

# Split dataseet for train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   cattle_breed                360 non-null    int64  
 1   health_status               360 non-null    int64  
 2   status                      360 non-null    int64  
 3   feeding_amount_KG_morning   300 non-null    float64
 4   score_morning               300 non-null    float64
 5   feeding_amount_KG_noon      300 non-null    float64
 6   score_noon                  300 non-null    float64
 7   feeding_amount_KG_evening   300 non-null    float64
 8   score_evening               300 non-null    float64
 9   travel_distance_per_day_KM  360 non-null    float64
dtypes: float64(7), int64(3)
memory usage: 28.2 KB


In [11]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 5: Predict and evaluate the model
y_pred = model.predict(X_test)

# Step 6: Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 100.00%


In [19]:
import numpy as np

# Define the encoding mappings as dictionaries
encodings = {
    'cattle_breed': {'Australian Milking Zebu': 0, 'Ayrshire': 1, 'Friesian': 2, 'Jersey': 3, 'Lanka White': 4, 'Sahiwal': 5},
    'health_status': {'Healthy': 0, 'Sick': 1},
    'status': {'Breeding': 0, 'Bulls': 1, 'Calves ': 2, 'Heifers ': 3, 'Lactating': 4, 'Pregnant': 5},
    'food_type_morning': {'Coconut Poonac': 0, 'Coconut Poonac, Grass': 1, 'Milk': 2},
    'food_type_noon': {'Coconut Poonac, Grass': 0, 'Grass, Paddy Straw': 1, 'Napier Grass, Guinea grass': 2,
                       'Napier Grass, Guinea grass, Para grass': 3, 'Napier Grass, Guinea grass,Gliricidia': 4,
                       'Paddy Straw, Grass (Chopped)': 5, 'Para grass, Gliricidia': 6},
    'food_type_evening': {'Milk': 0, 'Paddy Straw': 1, 'Paddy Straw, Corn': 2, 'Paddy Straw, Grass': 3, 'Paddy Straw, Legumes': 4}
}

# Method to encode input values and make a prediction
def predict_food_type(cattle_breed, health_status, status,
                       feeding_amount_KG_morning, score_morning, feeding_amount_KG_noon, score_noon,
                       feeding_amount_KG_evening, score_evening, travel_distance_per_day_KM):

    # Encode categorical input values based on the provided encodings
    encoded_input = [
        encodings['cattle_breed'].get(cattle_breed, -1),
        encodings['health_status'].get(health_status, -1),
        encodings['status'].get(status, -1),
        feeding_amount_KG_morning,  # Keep the numeric values as is
        score_morning,
        feeding_amount_KG_noon,
        score_noon,
        feeding_amount_KG_evening,
        score_evening,
        travel_distance_per_day_KM
    ]

    # If any value is not found in the encoding dictionary, return an error message
    if -1 in encoded_input[:6]:  # Check only encoded fields for invalid values
        return "Invalid input value(s)"

    # Convert the input into a 2D array (as required by the model)
    encoded_input = np.array(encoded_input).reshape(1, -1)

    # Predict using the trained model (assuming `model` is already trained)
    prediction = model.predict(encoded_input)

    # Return the predicted food type (the pattern)
    return prediction[0]

# Example usage
cattle_breed = 'Friesian'
health_status = 'Healthy'
status = 'Lactating'
feeding_amount_KG_morning = 5.0
score_morning = 8.0
feeding_amount_KG_noon = 6.0
score_noon = 7.5
feeding_amount_KG_evening = 4.5
score_evening = 7.0
travel_distance_per_day_KM = 12.0

predicted_food_type = predict_food_type(cattle_breed, health_status, status, feeding_amount_KG_morning, score_morning, feeding_amount_KG_noon,
                                         score_noon, feeding_amount_KG_evening, score_evening, travel_distance_per_day_KM)
print(f"Predicted Food Type Pattern: {predicted_food_type}")


Predicted Food Type Pattern: 0-2-4


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [20]:
def predict_food_type(cattle_breed, health_status, status,
                       feeding_amount_KG_morning, score_morning, feeding_amount_KG_noon, score_noon,
                       feeding_amount_KG_evening, score_evening, travel_distance_per_day_KM):

    # Encode categorical input values based on the provided encodings
    encoded_input = [
        encodings['cattle_breed'].get(cattle_breed, -1),
        encodings['health_status'].get(health_status, -1),
        encodings['status'].get(status, -1),
        feeding_amount_KG_morning,  # Keep the numeric values as is
        score_morning,
        feeding_amount_KG_noon,
        score_noon,
        feeding_amount_KG_evening,
        score_evening,
        travel_distance_per_day_KM
    ]

    # If any value is not found in the encoding dictionary, return an error message
    if -1 in encoded_input[:6]:  # Check only encoded fields for invalid values
        return "Invalid input value(s)"

    # Convert the input into a 2D array (as required by the model)
    encoded_input = np.array(encoded_input).reshape(1, -1)

    # Predict using the trained model (assuming `model` is already trained)
    prediction = model.predict(encoded_input)

    # Extract the predicted values for morning, noon, and evening food types
    morning_pred = prediction[0].split('-')[0]
    noon_pred = prediction[0].split('-')[1]
    evening_pred = prediction[0].split('-')[2]

    # Map these predicted values to food types using the dictionaries
    food_type_morning = [key for key, value in encodings['food_type_morning'].items() if value == int(morning_pred)][0]
    food_type_noon = [key for key, value in encodings['food_type_noon'].items() if value == int(noon_pred)][0]
    food_type_evening = [key for key, value in encodings['food_type_evening'].items() if value == int(evening_pred)][0]

    # Return the result as an object (dictionary)
    return {
        'morning': food_type_morning,
        'noon': food_type_noon,
        'evening': food_type_evening
    }

# Example usage
cattle_breed = 'Friesian'
health_status = 'Healthy'
status = 'Lactating'
feeding_amount_KG_morning = 5.0
score_morning = 8.0
feeding_amount_KG_noon = 6.0
score_noon = 7.5
feeding_amount_KG_evening = 4.5
score_evening = 7.0
travel_distance_per_day_KM = 12.0

predicted_food_type = predict_food_type(cattle_breed, health_status, status, feeding_amount_KG_morning, score_morning, feeding_amount_KG_noon,
                                         score_noon, feeding_amount_KG_evening, score_evening, travel_distance_per_day_KM)
print(f"Predicted Food Type: {predicted_food_type}")


Predicted Food Type: {'morning': 'Coconut Poonac', 'noon': 'Napier Grass, Guinea grass', 'evening': 'Paddy Straw, Legumes'}


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [21]:
import joblib
joblib.dump(model, 'feeding_pattern_model.joblib')

['feeding_pattern_model.joblib']